# Start

Distributed Version Control System:

People can all view different versions without worrying the central server malfunctions. 

Unlike other version control: delta-based, which records difference. 

Git takes snapshots of all versions. Git hardly deletes data, but only adds. One commit one snapshot. 

# Git Fundamentals

## Obtain Git Repository

1. Change a local location into a Git repository (windows):

    cd /c/user/my_project
    git init

2. Clone an existing Git repository

    git clone https://github.com/repository-name alias


## Update Repository

4 status: Untracked, Unmodified, Modified, Staged. 

.gitignore file defines files that don't need to be pushed. 

    # 忽略所有的 .a 文件
    *.a

    # 但跟踪所有的 lib.a，即便你在前面忽略了 .a 文件
    !lib.a

    # 只忽略当前目录下的 TODO 文件，而不忽略 subdir/TODO
    /TODO

    # 忽略任何目录下名为 build 的文件夹
    build/

    # 忽略 doc/notes.txt，但不忽略 doc/server/arch.txt
    doc/*.txt

    # 忽略 doc/ 目录及其所有子目录下的 .pdf 文件
    doc/**/*.pdf

## Log History

    $ git log --pretty=format:"%h - %an, %ar : %s"
    ca82a6d - Scott Chacon, 6 years ago : changed the version number
    085bb3b - Scott Chacon, 6 years ago : removed unnecessary test
    a11bef0 - Scott Chacon, 6 years ago : first commit

Junio Hamano 在 2008 年 10 月其间， 除了合并提交之外的哪一个提交修改了测试文件

    $ git log --pretty="%h - %s" --author='Junio C Hamano' --since="2008-10-01" \
    --before="2008-11-01" --no-merges -- t/
    5610e3b - Fix testcase failure when extended attributes are in use
    acd3b9e - Enhance hold_lock_file_for_{update,append}() API
    f563754 - demonstrate breakage of detached checkout with symbolic link HEAD
    d1a43f2 - reset --hard/read-tree --reset -u: remove unmerged new paths
    51a94af - Fix "checkout --track -b newbranch" on detached HEAD
    b0ad11e - pull: allow "git pull origin $something:$current_branch" into an unborn branch  

## Revoke

    $ git commit -m 'initial commit'
    $ git add forgotten_file
    $ git commit --amend
    
In this case there will only be the second commit. 

    git checkout -- <file>

Will recover the file from last commit. 

## Remote Repository

There can be multiple. Push automatically goes to the original one. 

To add a new one:

    git remote add <shortname> <url> 

To get whatever this current repository doesn't have that the other repository does:

    git fetch <remote>

## Tag

2 types of tag: lightweight and annotated. 

Annotated:

    git tag -a v1.4 -m "my version 1.4"

Lightweight:

    git tag v1.4

If one forgets to tag a previous version:

    git tag -a v1.2 9fceb02

The version to be added can be found:

    git log --pretty=oneline

It's not necessary to always include the full version name

Tag names will not be pushed to remote repository. Use:

    git push origin v1.5

Or more tags at the same time:

    git push origin --tags

To delete tags:

    git push origin --delete <tagname>

# Branching

Multi-verse. Each universe doesn't affect others. To create (not automatically swtich):

    git branch <branch_name>

To switch:

    git switch <branch_name>

To create and switch:

    git checkout -b <branch_name>

After that we do the same. Save, commit, push. It doesn't affect other branches. 

To merge, first switch to the main file, and then merge. 

    git merge <branch_name>

Then delete (the history is still stored):

    git branch -d <branch_name>

When there's a conflict, it's usually due to the change of the same file. 
1. Check conflict file
2. Manual edit and solve conflict. Difference will be shown
3. Add to stage
4. Push

Some principles:

    feature: new feature
    bugfix: solve bugs
    hotfix: emergency repair

Constantly synchronize. 

Use the following to check history:

    git log --oneline --decorate --graph --all 

Check which branch has already been merged:

    $ git branch --merged
    iss53
    * master

## Remote Branching

There could be branches on the cloud side. Those branches cannot be simply switched to. Use

    git branch -r

to check. The result can be:

    origin/main
    origin/feature-login

To switch to the second:

    git checkout -b feature-login origin/feature-login

git push doesn't apply to all remotes, only to designated ones. 

    git push <remote> <branch>

## Rebase

Change a tree into a stick. 

    $ git checkout experiment
    $ git rebase master

    master:  C1 --- C2 --- C3
                    \
    experiment:        C4


    git checkout experiment
    git rebase master

    master:    C1 --- C2 --- C3
                                \
    experiment:                   C4'   (现在 experiment 基于 C3)


    git checkout master
    git merge experiment   # 会是 fast-forward，master -> C4'

    master: C1 --- C2 --- C3 --- C4' 


Theoretically C4' is identical with the result of merge. 

**Interactive Rebase**

    git checkout feature
    git rebase -i master

The output:

    pick a1b2c3 fix typo
    pick d4e5f6 add tests
    pick 9a0b1c refactor method

Change pick to:

squash: That specific commit will be combined with the previous commit
drop: That specific commit will be dropped


    $ git rebase --onto master server client

It means take client branch, find the patch after server, and put it on master. Then:

    $ git checkout master
    $ git merge client

# Protocol

To avoid push and pull, build a mutual storage everyone can access: a Git server. There needs to be protocols. 

1. Local protocol

A shared document folder. Everything will be local. 

2. HTTP

3. Secure Shell

4. Git

# Distributed Git

**Two Developers**

There are two repository for everyone, one local (master) and one remote (origin/master). Two people each clone the main repository to their local folder, hence the local master. 

Jessica committed a change and pushed it from master to origin/master. Now origin/master is the same as Jessica's master. Now John tries to commit and push but fails. So he needs to fetch origin, and get what Jessica did. This changes the origin/master of John. Note that John's origin/master will not change because of Jessica's push. Because John's origin/master is still the status of last fetch or clone. It doesn't change itself. 

After that, John merge origin/master and absorb the change onto his master. In order to update John's change, he then needs to push origin master. 

Meanwhile, Jessica made 3 commits in her new branch issue54. She learnt that John pushed some work, so she fetch origin. To check what needs to be merged, she calls:

    git log --no-merges issue54..origin/master

Then she goes back to her master, either first merges newly fetched origin/master or issue54. Eventually she pushes the final master: 

    git push origin master

Difference between **fetch** and **pull**: 

pull = fetch + merge

fetch changes local remote repository, but not local master. pull changes both. 

**A Larger Group**

Jessica works on featureA with John, on featureB with Josie. She first checkout -b featureA and commits a change. She doesn't have authoritation to push the main branch master, but she can push featureA, as she

    git push -u origin featureA

u for upstream, so that keeps track that the upstream of featureA is origin/featureA. In the future she can simply use pull, without specifying names. Then she starts to build a new branch:

    git checkout -b featureB origin/master

the origin/master is to make sure the branch is based on the latest remote master, rather than her local old master. She then makes some commits. Before she pushes, she learns that the original work of featureB has been pushed to featureBee. So Jessica first fetch origin and merge origin/featureBee. She doesn't want all featureB to be pushed to the server, so she pushes the latest to featureBee:

    git push -u origin featureB:featureBee
    git push -u <remote> <local branch>:<remote branch>

Now John tells Jessica there's change in featureA. So Jessica fetch origin and checks the log:

    git log featureA..origin/featureA

She likes it and merges it with hers:

    git checkout featureA
    git merge origin/featureA
    git push

**Open Souce**

It's impossible to modify the repository, so we sue fork, which copies a remote repository. We change in fork and eventually pull request. 

    git clone <url>      # 克隆主项目
    cd project
    git checkout -b featureA

It's recommended to modify in a local branch. After that we make some commits, and fork the repository. Add the fork as a remote repository:

    git remote add myfork <url>
    git push -u myfork featureA

Request a modification in the owner's repository:

    git request-pull origin/master myfork

If the owner's repository has already changed, we need to rebase:

    git checkout featureA
    git fetch origin
    git rebase origin/master

This way we change our version to after the latest one. Then force push:

    git push -f myfork featureA

If the owner gives feedbacks, we make a new branch:

    git checkout -b featureAv2 origin/master
    git merge --squash featureA

which compress all commits of featureA. We commit and push myfork featureAv2. 


## Maintaining a project

 For email patch:

    git apply --check xxx.patch
    git apply xxx.patch

For Git format-patch mbox file:

    git am xxx.patch

If error, solve it manually and:

    git am --resolved

Or:

    git am -3 xxx.patch


If Jessica said there's a good function in her branch ruby-client, we can add it as a remote branch:

    $ git remote add jessica git://github.com/jessica/myproject.git
    $ git fetch jessica
    $ git checkout -b rubyclient jessica/ruby-client

So in the future if there's any change we can use fetch and checkout. 

## Cherry-Pick

Pick only specific ones, rather than the whole chain. 

    A -- B -- C -- D -- E        （feature）
    \
      X -- Y                      （master）

    git checkout master
    git cherry-pick D

    A -- B -- C -- D -- E        （feature）
    \
      X -- Y -- D'               （master）


It's used when something urgent is required from feature. 

## Rerere

Same solution for same merge conflict in the future. 

    $ git config --global rerere.enabled true

# SSH

Secure Shell: it's used to safely connect the remote computer, e.g. the server of GitHub. 

It's used for Git manipulations like push, pull. 

Like https which requires password. 

# Git Tools

## Revision Selection

    git show HEAD      # 当前提交
    git show HEAD^     # 上一个提交
    git show HEAD~2    # 往前第 2 个提交

    git show HEAD@{yesterday}   # HEAD 昨天的位置
    git show HEAD@{2.days.ago}  # 2 天前的位置

reflog to show the journey of HEAD
    
    git reflot
    git reflog show master

    HEAD~3^2   → HEAD 的第 3 代祖先的第二父提交


The commit (version, e.g. c3, c4 is a sub-branch, then the commits after c2 on the other branch will be retrieved) in B but not in A:

    git log master..featureA

The commit in refA or refB but not in refC

    git log refA refB ^refC

The commit in A or B but not mutual:

    git log master...experiment --left-right

The output will be like:

    < F
    < E
    > D
    > C


## Stash

If you don't want to commit because you haven't finished, yet you have to switch to another branch: use stash

    git stash push -m "work in progess"

Check what's stashed:

    git stash list

Get the latest stashed back to workspace:

    git stash apply

Get the latest stashed back to workspace but delete stash:

    git stash pop

Delete stash:

    git stash drop stash@{0}
    git stash clear

Save the change in stash:

    git stash --keep-index

## Clean

Sometimes there are files automatically generated and have ?? before the name of the file when git status. These untracked files can be deleted if:

    git clean -d -n. 

## Signature

Basic signing:

    git commit --author="John Doe <johndoe@email.com>

But everyone can write this line and claim to be the person. 

GPG: GNU Privacy Guard

Private Key: On personal laptop, stores the sign

Public Key: Public, so that other people know it's you

When tagging using:

    git tag -s v1.0 -m "my signed release"

Other people can test it:

    git tag -v v1.0

If the output is good signature then it's valid. 

When committing:

    git commit -S -m "fix bug"

In git log --show-signature, we can see gpg: Good signature from "name <email>". Or 

    git merge -S verified-feature

Establishment:

1. pip install gnupg

2. gpg --gen-key

3. gpg --list-keys # to check id

4. git config --global user.signingkey <id> # from step 3

5. git config --global commit.gpgSign true

6. gpg --armor --export you@example.com > mykey.asc

so that people can check if they import:

gpg --import mykey.asc

# Configuration

## Basic Configuration

    $ git config --global user.name "John Doe"
    $ git config --global user.email johndoe@example.com

## Hook

Client-Side Hook: Committing-workflow, email, and other hooks. (Hooks are not copied when cloning)

**Commiting-Workflow**

    .git/hooks/
    ├── pre-commit.sample
    ├── commit-msg.sample
    ├── post-commit.sample
    ...

To use them just drop "sample".

Pre-commit: Check validity or style

    cp .git/hooks/pre-commit.sample .git/hooks/pre-commit


**Email**

First commit, then 'git format-patch origin/main`, which generates a patch file.

Set email info:

    git config --global sendemail.smtpserver smtp.gmail.com
    git config --global sendemail.smtpuser yourname@gmail.com
    git config --global sendemail.smtpencryption tls
    git config --global sendemail.smtpserverport 587
    git config --global sendemail.confirm always

To send email: git send-email 0001-Fix-bug-in-input-parsing.patch \
  --to maintainer@example.com

To use hook:


    #!/bin/sh
    # 自动生成并发送补丁邮件
    last_commit=$(git rev-parse HEAD)
    patch_file="/tmp/$(git format-patch -1 $last_commit --stdout > /tmp/last.patch && echo last.patch)"

    echo "📦 Sending patch for commit: $last_commit"
    git send-email "$patch_file" \
    --to maintainer@example.com \
    --suppress-cc=all --quiet

And set:

    chmod +x .git/hooks/post-commit

So that every time there's a commit there will be an email. 